In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd

# Paths to your datasets
train_path = '/content/drive/My Drive/train.csv'
test_path = '/content/drive/My Drive/test.csv'




Mounted at /content/drive


In [ ]:
# Import essential libraries
import pandas as pd
import numpy as np
import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC


In [ ]:
# Load datasets
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

# Inspect data
print(train_df.head())
print(test_df.head())


                                category                       sub_category  \
0  Online and Social Media Related Crime  Cyber Bullying  Stalking  Sexting   
1                 Online Financial Fraud                  Fraud CallVishing   
2               Online Gambling  Betting           Online Gambling  Betting   
3  Online and Social Media Related Crime                   Online Job Fraud   
4                 Online Financial Fraud                  Fraud CallVishing   

                                  crimeaditionalinfo  
0  I had continue received random calls and abusi...  
1  The above fraudster is continuously messaging ...  
2  He is acting like a police and demanding for m...  
3  In apna Job I have applied for job interview f...  
4  I received a call from lady stating that she w...  
                                    category  \
0  RapeGang Rape RGRSexually Abusive Content   
1                     Online Financial Fraud   
2             Cyber Attack/ Dependent Crimes   
3  

In [ ]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Download necessary NLTK data
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# Initialize lemmatizer and stop words
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Function for preprocessing
def preprocess_text(text):
    # Check if the text is a string; if not, convert it to an empty string
    text = str(text) if isinstance(text, str) else ""
    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Tokenize and remove stop words
    tokens = word_tokenize(text.lower())
    tokens = [word for word in tokens if word not in stop_words]
    # Lemmatize words
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)

# Apply preprocessing to 'crimeaditionalinfo' in both train and test datasets
train_df['cleaned_text'] = train_df['crimeaditionalinfo'].apply(preprocess_text)
test_df['cleaned_text'] = test_df['crimeaditionalinfo'].apply(preprocess_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
# Initialize TF-IDF Vectorizer
tfidf = TfidfVectorizer(max_features=5000)
X_train = tfidf.fit_transform(train_df['cleaned_text']).toarray()
X_test = tfidf.transform(test_df['cleaned_text']).toarray()

# Separate target labels for both category and sub_category
y_train = train_df[['category', 'sub_category']]
y_test = test_df[['category', 'sub_category']]


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import MultinomialNB

# Convert to DataFrames to handle NaN values
X_train = pd.DataFrame(X_train)
y_train = pd.DataFrame(y_train, columns=['category', 'sub_category'])
X_test = pd.DataFrame(X_test)
y_test = pd.DataFrame(y_test, columns=['category', 'sub_category'])

# Fill NaN values
X_train = X_train.fillna('')  # Replace NaNs with an empty string for text data
y_train = y_train.fillna(y_train.mode().iloc[0])  # Replace NaNs in y_train with the mode value
X_test = X_test.fillna('')
y_test = y_test.fillna(y_test.mode().iloc[0])

# Convert back to arrays for model compatibility
X_train = X_train.values
y_train = y_train.values
X_test = X_test.values
y_test = y_test.values

# Initialize and train the MultiOutputClassifier with Naive Bayes
nb_model = MultiOutputClassifier(MultinomialNB())
nb_model.fit(X_train, y_train)

# Predict on the test set
y_pred_nb = nb_model.predict(X_test)

# Extract predictions for 'category' and 'sub_category'
y_pred_category_nb = y_pred_nb[:, 0]
y_pred_sub_category_nb = y_pred_nb[:, 1]

# Evaluate for 'category'
print("Naive Bayes Model - Category")
print("Accuracy:", accuracy_score(y_test[:, 0], y_pred_category_nb))
print("Precision:", precision_score(y_test[:, 0], y_pred_category_nb, average='weighted'))
print("Recall:", recall_score(y_test[:, 0], y_pred_category_nb, average='weighted'))
print("F1 Score:", f1_score(y_test[:, 0], y_pred_category_nb, average='weighted'))
print("\nCategory Classification Report:\n", classification_report(y_test[:, 0], y_pred_category_nb))

# Evaluate for 'sub-category'
print("\nNaive Bayes Model - Sub-category")
print("Accuracy:", accuracy_score(y_test[:, 1], y_pred_sub_category_nb))
print("Precision:", precision_score(y_test[:, 1], y_pred_sub_category_nb, average='weighted'))
print("Recall:", recall_score(y_test[:, 1], y_pred_sub_category_nb, average='weighted'))
print("F1 Score:", f1_score(y_test[:, 1], y_pred_sub_category_nb, average='weighted'))
print("\nSub-category Classification Report:\n", classification_report(y_test[:, 1], y_pred_sub_category_nb))


Naive Bayes Model - Category
Accuracy: 0.7368791828108489


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Precision: 0.7027037071980846
Recall: 0.7368791828108489
F1 Score: 0.7103998632107116


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Category Classification Report:
                                                       precision    recall  f1-score   support

                               Any Other Cyber Crime       0.36      0.29      0.32      3670
Child Pornography CPChild Sexual Abuse Material CSAM       0.71      0.12      0.21       123
                      Crime Against Women & Children       0.00      0.00      0.00         4
                                Cryptocurrency Crime       0.77      0.14      0.24       166
                      Cyber Attack/ Dependent Crimes       1.00      1.00      1.00      1261
                                     Cyber Terrorism       0.00      0.00      0.00        52
      Hacking  Damage to computercomputer system etc       0.37      0.15      0.21       592
                            Online Cyber Trafficking       0.00      0.00      0.00        61
                              Online Financial Fraud       0.83      0.90      0.87     18896
                         

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Precision: 0.49834111095378053
Recall: 0.49540491210093185
F1 Score: 0.4747697361653262


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Sub-category Classification Report:
                                                                       precision    recall  f1-score   support

                             Business Email CompromiseEmail Takeover       0.00      0.00      0.00        90
                                           Cheating by Impersonation       0.11      0.00      0.00       719
                                        Computer Generated CSAM/CSEM       0.00      0.00      0.00         2
                                                Cryptocurrency Fraud       0.74      0.16      0.26       166
                                    Cyber Blackmailing & Threatening       0.00      0.00      0.00         1
                                   Cyber Bullying  Stalking  Sexting       0.40      0.68      0.51      1366
                                                     Cyber Terrorism       0.00      0.00      0.00        52
                             Damage to computer computer systems etc       0.00  

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import pickle

# Save the model to a file
with open('naive_bayes_model.pkl', 'wb') as file:
    pickle.dump(nb_model, file)
with open('naive_bayes_model.pkl', 'rb') as file:
    nb_model = pickle.load(file)
drive.mount('/content/drive')

# Define the path to save the model in your Google Drive
model_path = '/content/drive/My Drive/Colab Notebooks/nb_model.pkl'

# Save the model to Google Drive
with open(model_path, 'wb') as file:
    pickle.dump(nb_model, file)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
